## Translation Inference using Online Endpoints

This sample shows how deploy `translation` type models to an online endpoint for inference.

### Task
`translation` converts a sequence of text from one language to another. It is one of several tasks you can formulate as a sequence-to-sequence problem, a powerful framework for returning some output from an input, like translation or summarization. `translation` systems are commonly used for translation between different language texts, but it can also be used for speech or some combination in between like text-to-speech or speech-to-text.

### Model
Models that can perform the `translation` task are tagged with `task: translation`. We will use the `t5-small` model in this notebook. If you opened this notebook from a specific model card, remember to replace the specific model name. If you don't find a model that suits your scenario or domain, you can discover and [import models from HuggingFace hub](../../import/import-model-from-huggingface.ipynb) and then use them for inference. 

### Inference data
We will use the [wmt16 (ro-en)](https://huggingface.co/datasets/wmt16) dataset. A copy of this dataset is available in the [wmt16-en-ro-dataset](./wmt16-en-ro-dataset/) folder. 

### Outline
* Setup pre-requisites.
* Pick a model to deploy.
* Prepare data for inference. 
* Deploy the model for real time inference.
* Test the endpoint
* Clean up resources.

### 1. Setup pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, ClientSecretCredential
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

workspace_ml_client = MLClient(
        credential,
        subscription_id =  "<SUBSCRIPTION_ID>",
        resource_group_name =  "<RESOURCE_GROUP>",
        workspace_name =  "<WORKSPACE_NAME>"
)
# the models, fine tuning pipelines and environments are available in the AzureML system registry, "azureml-preview"
registry_ml_client = MLClient(credential, registry_name="azureml-preview")

# genrating a unique timestamp that can be used for names and versions that need to be unique
timestamp = str(int(time.time())) 


### 2. Pick a model to deploy

Browse models in the Model Catalog in the AzureML Studio, filtering by the `translation` task. In this example, we use the `t5-small` model. If you have opened this notebook for a different model, replace the model name and version accordingly. 

In [ ]:
model_name = "t5-small"
model_version = "4"
foundation_model=registry_ml_client.models.get(model_name, model_version)
print ("\n\nUsing model name: {0}, version: {1}, id: {2} for inferencing".format(foundation_model.name, foundation_model.version, foundation_model.id))

### 3. Prepare data for inference.

A copy of the wmt16-en-ro dataset is available in the [wmt16-en-ro-dataset](./wmt16-en-ro-dataset/) folder.  The next few cells show basic data preparation:
* Visualize some data rows
* Save few samples in the format that can be passed as input to the online-inference endpoint.

In [ ]:
# load the ./wmt16-en-ro-dataset/train.jsonl file into a pandas dataframe and show the first 5 rows
import pandas as pd
pd.set_option('display.max_colwidth', 0) # set the max column width to 0 to display the full text
train_df = pd.read_json("./wmt16-en-ro-dataset/train.jsonl", lines=True)
train_df.head()

### 4. Deploy the model to an online endpoint
Online endpoints give a durable REST API that can be used to integrate with applications that need to use the model.

In [ ]:
import time, sys
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, OnlineRequestSettings

# Create online endpoint - endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
timestamp = int(time.time())
online_endpoint_name = "translation-" + str(timestamp)
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for " + foundation_model.name + ", for translation task",
    auth_mode="key"
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [ ]:
# create a deployment
demo_deployment = ManagedOnlineDeployment(
    name="demo",
    endpoint_name=online_endpoint_name,
    model=foundation_model.id,
    instance_type="Standard_DS2_v2",
    instance_count=1,
    request_settings=OnlineRequestSettings(
        request_timeout_ms=60000,
    ),
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {"demo": 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

### 5. Test the endpoint with sample data

We will fetch some sample data from the test dataset and submit to online endpoint for inference. We will then show the display the scored labels alongside the ground truth labels

In [ ]:
import json
import os
# read the ./wmt16-en-ro-dataset/train.jsonl file into a pandas dataframe 
df = pd.read_json("./wmt16-en-ro-dataset/train.jsonl", lines=True)
# escape single and double quotes in the text column
df["en"] = df["en"].str.replace("'", "\\'").str.replace('"', '\\"')
# pick 1 random row
sample_df=df.sample(1)
# create a json object with the key as "inputs" and value as a list of values from the en column of the sample_df dataframe
test_json = {"inputs": {"input_string": sample_df["en"].tolist()}, "parameters": {"task_type": "translation_en_to_ro"}}
# save the json object to a file named sample_score.json in the ./wmt16-en-ro-dataset folder
with open(os.path.join(".", "wmt16-en-ro-dataset","sample_score.json"), "w") as f:
    json.dump(test_json, f)
sample_df.head()

In [ ]:
# score the sample_score.json file using the online endpoint with the azureml endpoint invoke method
response=workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="demo",
    request_file="./wmt16-en-ro-dataset/sample_score.json"
)
print("raw response: \n", response, "\n")
# convert the json response to a pandas dataframe 
response_df = pd.read_json(response)
response_df.head()

In [ ]:
# compare the predicted translation with the ground truth translation
response_df.rename(columns={"translation_text": "predicted_translation"}, inplace=True)
response_df["ground_truth_translation"] = sample_df["ro"].tolist()
response_df.head()

### 6. Delete the online endpoint
Don't forget to delete the online endpoint, else you will leave the billing meter running for the compute used by the endpoint

In [ ]:
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()